<a href="https://colab.research.google.com/github/MirajMjn/AI_Collab_Repository/blob/main/anime_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anime Recommendation System

The following project attempts to make use of the [Anime Dataset](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020) to make a content based recommendation function implemented in a function such that the function takes in an anime and recommends similar anime.

Note that the dataset is not brand new.

**Submitted By : Miraj Maharjan (ACE077BEI021)**

**For completion of AI LAB (BEI IV/I)**

# Mounting Google Drive

Mounting google drive and changing the working directory for easier access

In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
cd /content/drive/MyDrive/AI_folder/Datasets/anime_data

/content/drive/MyDrive/AI_folder/Datasets/anime_data


# Importing Necessary Libraries

In [141]:
import pandas as pd
import numpy as np

# Loading The Dataset

In [142]:
df = pd.read_csv("anime.csv")
df.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


# Feature Selection

We make use of the following features :
- English Name
- Score
- Genres
- Type
- Episodes

In [143]:
# Extracting the features into a new sub dataset

anime = df[["English name", "Score", "Genres", "Type", "Episodes"]]

# Data Preprocessing and Cleaning

In [144]:
# to convert type object to type list so we can concatenate to make tags feature

def Convert(string):
    return (string.split(","))

anime["Genres"] = anime["Genres"].apply(Convert)
anime["Score"] = anime["Score"].apply(Convert)
anime["Type"] = anime["Type"].apply(Convert)
anime["Episodes"] = anime["Episodes"].apply(Convert)

<ipython-input-144-066b19d84fc7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime["Genres"] = anime["Genres"].apply(Convert)
<ipython-input-144-066b19d84fc7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime["Score"] = anime["Score"].apply(Convert)
<ipython-input-144-066b19d84fc7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [145]:
# Creating new features more relevant to us, namely the name and the tags

anime["anime_name"] = anime["English name"]

anime["tags"] = anime["Genres"]+anime["Type"]

<ipython-input-145-74cabab85f92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime["anime_name"] = anime["English name"]
<ipython-input-145-74cabab85f92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime["tags"] = anime["Genres"]+anime["Type"]


In [146]:
# The data set has "Unknown" entries that we want to get rid of, these are most likely anime with no English audience

anime["English name"].value_counts()

English name
Unknown                            10565
Meow Meow Japanese History             4
Rakshasa Street                        4
Cyborg 009                             4
Spirit Guardians                       4
                                   ...  
Gordian Warrior                        1
Master of Martial Hearts               1
Adrift in the Pacific                  1
Return of Pero                         1
Higurashi:When They Cry – SOTSU        1
Name: count, Length: 6831, dtype: int64

In [147]:
# Basically removing all entries whose english name is Unknown

anime = anime[anime["English name"] != 'Unknown']

In [148]:
# Verifying removal

anime["English name"].value_counts()

English name
Rakshasa Street                    4
Meow Meow Japanese History         4
Cyborg 009                         4
Spirit Guardians                   4
RIN-NE                             3
                                  ..
Gordian Warrior                    1
Master of Martial Hearts           1
Adrift in the Pacific              1
Return of Pero                     1
Higurashi:When They Cry – SOTSU    1
Name: count, Length: 6830, dtype: int64

# Creating the Concise data frame with just anime name and tags

In [149]:
feature_df = anime[["anime_name", "tags"]]

In [150]:
# This changes the 'tags' from list to space separated words

feature_df["tags"] = feature_df["tags"].apply(lambda x: " ".join(x))
feature_df.head()

<ipython-input-150-164b685120b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df["tags"] = feature_df["tags"].apply(lambda x: " ".join(x))


,anime_name,tags
0,Cowboy Bebop,Action Adventure Comedy Drama Sci-Fi Spac...
1,Cowboy Bebop:The Movie,Action Drama Mystery Sci-Fi Space Movie
2,Trigun,Action Sci-Fi Adventure Comedy Drama Shou...
3,Witch Hunter Robin,Action Mystery Police Supernatural Drama ...
4,Beet the Vandel Buster,Adventure Fantasy Shounen Supernatural TV


In [157]:
# making all anime names lower case for convenience in comparison

feature_df["anime_name"] = feature_df["anime_name"].str.lower()

<ipython-input-157-ba93a3f0e99d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df["anime_name"] = feature_df["anime_name"].str.lower()


## The final anime and feature_df data

In [152]:
anime

,English name,Score,Genres,Type,Episodes,anime_name,tags
0,Cowboy Bebop,[8.78],"[Action, Adventure, Comedy, Drama, Sci-Fi,...",[TV],[26],Cowboy Bebop,"[Action, Adventure, Comedy, Drama, Sci-Fi,..."
1,Cowboy Bebop:The Movie,[8.39],"[Action, Drama, Mystery, Sci-Fi, Space]",[Movie],[1],Cowboy Bebop:The Movie,"[Action, Drama, Mystery, Sci-Fi, Space, Mo..."
2,Trigun,[8.24],"[Action, Sci-Fi, Adventure, Comedy, Drama,...",[TV],[26],Trigun,"[Action, Sci-Fi, Adventure, Comedy, Drama,..."
3,Witch Hunter Robin,[7.27],"[Action, Mystery, Police, Supernatural, Dr...",[TV],[26],Witch Hunter Robin,"[Action, Mystery, Police, Supernatural, Dr..."
4,Beet the Vandel Buster,[6.98],"[Adventure, Fantasy, Shounen, Supernatural]",[TV],[52],Beet the Vandel Buster,"[Adventure, Fantasy, Shounen, Supernatural,..."
...,...,...,...,...,...,...,...
17512,Cooking with Valkyries Spring Festival Special,[6.83],"[Slice of Life, Comedy]",[Special],[1],Cooking with Valkyries Spring Festival Special,"[Slice of Life, Comedy, Special]"
17513,The Promised Neverland Season 2 Episode 5.5,[4.81],"[Mystery, Psychological, Supernatural, Thri...",[Special],[1],The Promised Neverland Season 2 Episode 5.5,"[Mystery, Psychological, Supernatural, Thri..."
17522,Immemorial Love For You 2nd Season,[Unknown],"[Fantasy, Romance]",[ONA],[16],Immemorial Love For You 2nd Season,"[Fantasy, Romance, ONA]"
17532,Da Shen Xian,[Unknown],"[Action, Martial Arts, Fantasy, Shounen]",[ONA],[16],Da Shen Xian,"[Action, Martial Arts, Fantasy, Shounen, ONA]"


In [153]:
feature_df

,anime_name,tags
0,cowboy bebop,Action Adventure Comedy Drama Sci-Fi Spac...
1,cowboy bebop:the movie,Action Drama Mystery Sci-Fi Space Movie
2,trigun,Action Sci-Fi Adventure Comedy Drama Shou...
3,witch hunter robin,Action Mystery Police Supernatural Drama ...
4,beet the vandel buster,Adventure Fantasy Shounen Supernatural TV
...,...,...
17512,cooking with valkyries spring festival special,Slice of Life Comedy Special
17513,the promised neverland season 2 episode 5.5,Mystery Psychological Supernatural Thriller...
17522,immemorial love for you 2nd season,Fantasy Romance ONA
17532,da shen xian,Action Martial Arts Fantasy Shounen ONA


# The actual AI model Bit

In [154]:
# The actual main bit

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(feature_df["tags"]).toarray()

# print(vector)

ps = PorterStemmer()

def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

feature_df["tags"] = feature_df["tags"].apply(stem)

# print(feature_df)

similarity = cosine_similarity(vector)
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]
# similarity[0]

<ipython-input-154-3936c03e81a2>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df["tags"] = feature_df["tags"].apply(stem)


[(752, 0.9428090415820634),
 (285, 0.9354143466934851),
 (910, 0.9354143466934851),
 (1272, 0.9354143466934851),
 (2, 0.8749999999999999)]

In [155]:
def recommend(name):
    name.lower() # make lowercase
    anime_index = feature_df[feature_df["anime_name"] == name].index[0]
    distances = similarity[anime_index]
    anime = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in anime:
        print(feature_df.iloc[i[0]].anime_name)


# Output : Enter name of an anime

In [156]:
recommend('berserk')

vampire hunter d
neo tokyo
garaga
vexille
armageddon
